## Importing Libraries and Inital Setup


In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [ ]:
import pandas as pd
import pyomo.environ as pyo

# Create a solver engine to use
opt = pyo.SolverFactory('cbc')

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')
dir_path = '/content/drive/MyDrive/Advanced Analytics - Fast Fashion Supply Chain Optimization'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Convert CSV files to DataFrame
log_data = pd.read_csv(f'{dir_path}/Log Data.csv')
products = pd.read_csv(f'{dir_path}/Products.csv')
warehouse_df = pd.read_csv(f'{dir_path}/Warehouse Shipping Costs.csv')
production_costs = pd.read_csv(f'{dir_path}/Production Costs.csv')

## Data Preprocessing and Cleaning

In [ ]:
# Convert 'Date' column to datetime
log_data['Date'] = pd.to_datetime(log_data['Date'])

# Extract year and month for grouping
log_data['Year-Month'] = log_data['Date'].dt.to_period('M')
# Remove factory 4
log_data = log_data[log_data[' Source Factory'] != 'F004']
log_data.reset_index(drop=True, inplace=True)

## Creating Monthly Analysis tables for Factory, Warehouse and Products

In [ ]:
# Group by 'Source Factory' and 'Year-Month', summing up 'Total No. of Pieces'
monthly_pieces = log_data.groupby([' Source Factory', 'Year-Month'])['Total No. of Pieces'].sum().unstack(fill_value=0)
monthly_pieces.reset_index(inplace=True)

#Adding a new column to display the average
monthly_pieces['Average'] = monthly_pieces.iloc[:, 1:].mean(axis=1)

# Display table
monthly_pieces.head()

Year-Month,Source Factory,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,Average
0,F001,2870679,2945895,2797910,2773265,2740840,2704991,2829022,2737816,2972655,2346308,2771938.1
1,F002,2085957,2067006,2012840,2075232,2142032,1895091,1939513,1954519,2232405,1754122,2015871.7
2,F003,2183906,2324811,2332787,2132990,2099759,2411203,2307023,2088129,2061824,1694893,2163732.5
3,F005,2994017,3126821,3279032,3224120,3315401,3062885,3241330,3054107,3221776,2578979,3109846.8


In [ ]:
# Group by 'Warehouse' and 'Year-Month', summing up 'Total No. of Pieces'
warehouse_monthly_pieces = log_data.groupby(['Dest. Warehouse', 'Year-Month'])['Total No. of Pieces'].sum().unstack(fill_value=0)
warehouse_monthly_pieces.reset_index(inplace=True)

#Adding a column to display the maximum capacity for each Warehouse for all the months
warehouse_monthly_pieces['Maximum'] = warehouse_monthly_pieces.iloc[:, 1:].max(axis=1)

# Rename the column in the DataFrame
warehouse_monthly_pieces.rename(columns={'Dest. Warehouse': 'Warehouse_ID'}, inplace=True)
warehouse_monthly_pieces.head()

Year-Month,Warehouse_ID,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,Maximum
0,W001,479266,529721,507808,506186,492131,515465,482244,491227,512755,387120,529721
1,W002,518039,506231,505169,499587,527855,513591,527653,502450,536115,417047,536115
2,W003,500410,472563,501030,479303,478898,469018,499238,474874,472377,410583,501030
3,W004,528930,532179,562589,522226,520867,540374,533951,508639,538779,460763,562589
4,W005,481588,512134,496687,497148,523957,500265,521455,491171,503341,436568,523957


In [ ]:
# Group by 'Product_ID' and 'Year-Month', summing up 'No. of Pieces sold'
demand_df = log_data.groupby(['Product_ID', 'Year-Month'])['No. of pieces sold'].sum().unstack(fill_value=0)

# Reset index for a cleaner table
demand_df.reset_index(inplace=True)

# Display the average that will be used for further analysis
demand_df['Average'] = demand_df.iloc[:, 1:].mean(axis=1)

demand_df.head()

Year-Month,Product_ID,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,Average
0,P001,502067,554413,528445,418975,384925,532175,534910,514669,406125,297169,467387.3
1,P002,345969,408383,543187,409705,388611,539812,464751,361271,378297,311774,415176.0
2,P003,371081,372841,373192,484928,532200,347049,391064,361495,371749,298311,390391.0
3,P004,493331,466589,353363,358762,359951,333276,351700,343016,343610,296301,369989.9
4,P005,366762,344521,383876,376973,411126,517745,488425,349091,381127,284263,390390.9


We don't have data for all the combinations of factories and products as some factories might not manufacture particular products. Therefore we will create a separate dataframe for all the combinations and then map the manufacturing costs for the ones we have. Wherever we don't have data for a combination, we will substitue the value by a high number (999 in this case) so that the solver doesn't consider it to compute the optimal solution.  

In [ ]:
# Define the parameters
factory_ids = ['F001', 'F002', 'F003', 'F005'] #only the ones we are using (excluding Factory F004)
product_ids = [f'P{i:03}' for i in range(1, 41)]  # P001 to P040

# Create the DataFrame for products and factories
rows = []
for factory in factory_ids:
    for product in product_ids:
        rows.append({'Factory_ID': factory, 'Product_ID': product, 'Manufac_Cost': None})

production_costs_demo = pd.DataFrame(rows)

# Create a dictionary
cost_dict = production_costs.set_index(['Factory_ID', 'Product_ID'])['Manufac_Cost'].to_dict()

# Update Manufac_Cost in production_cost_demo
production_costs_demo['Manufac_Cost'] = production_costs_demo.apply(
    lambda row: cost_dict.get((row['Factory_ID'], row['Product_ID']), 999), axis=1
)

# Display the updated DataFrame
production_costs = production_costs_demo
production_costs

,Factory_ID,Product_ID,Manufac_Cost
0,F001,P001,9.456023
1,F001,P002,10.815965
2,F001,P003,15.966598
3,F001,P004,10.307926
4,F001,P005,8.156190
...,...,...,...
155,F005,P036,999.000000
156,F005,P037,14.584286
157,F005,P038,999.000000
158,F005,P039,5.611075


## Warehouse Shipping Analysis



Again we don't have data for all the combinations of Factories, Products and Warehouses. We will create a new Dataframe and then map the original data to this. We will again replace the missing combinations with a high value to make sure its not picked to produce an optimal solution.

In [ ]:
# Divide the 'Shipping Cost (per 1000 pieces)' column by 1000 to get the individual shipping costs
warehouse_df['Shipping Cost (per 1000 pieces)'] /= 1000

# Define the parameters
warehouse_ids = [f'W{i:03}' for i in range(1, 21)]  # W001 to W020
source_factory_ids = ['F001', 'F002', 'F003', 'F005'] # excluding F004
product_ids = [f'P{i:03}' for i in range(1, 41)]  # P001 to P040

# Create the DataFrame
rows = []
for warehouse in warehouse_ids:
    for factory in source_factory_ids:
        for product in product_ids:
            rows.append({
                'Warehouse_ID': warehouse,
                'Source_Factory_ID': factory,
                'Product_ID': product,
                'Shipping Cost': None #To be mapped with the original table later
            })

shipping_costs_df = pd.DataFrame(rows)

# Display the first few rows and shape of the DataFrame
print(shipping_costs_df.head())
print(f"\nShape of the DataFrame: {shipping_costs_df.shape}")

  Warehouse_ID Source_Factory_ID Product_ID Shipping Cost
0         W001              F001       P001          None
1         W001              F001       P002          None
2         W001              F001       P003          None
3         W001              F001       P004          None
4         W001              F001       P005          None

Shape of the DataFrame: (3200, 4)


In [ ]:
# Create a dictionary from the warehouse dataframe
shipping_dict = warehouse_df.set_index(['Warehouse_ID', 'Source Factory_ID', 'Product_ID'])['Shipping Cost (per 1000 pieces)'].to_dict()

# Update Shipping_Cost in shipping_costs_df
shipping_costs_df['Shipping Cost'] = shipping_costs_df.apply(
    lambda row: shipping_dict.get((row['Warehouse_ID'], row['Source_Factory_ID'], row['Product_ID']), 999),
    axis=1
)

# Display the first few rows of the updated DataFrame
print(shipping_costs_df.head(10))

  Warehouse_ID Source_Factory_ID Product_ID  Shipping Cost
0         W001              F001       P001          999.0
1         W001              F001       P002          999.0
2         W001              F001       P003          999.0
3         W001              F001       P004          999.0
4         W001              F001       P005          999.0
5         W001              F001       P006            1.8
6         W001              F001       P007          999.0
7         W001              F001       P008            1.0
8         W001              F001       P009          999.0
9         W001              F001       P010          999.0


In [ ]:
from pyomo.environ import *

# Initialize Data
unique_products = [f'P{str(i).zfill(3)}' for i in range(1, 41)]
unique_factories = ['F001', 'F002', 'F003', 'F005']
unique_warehouses = [f'W{str(i).zfill(3)}' for i in range(1, 21)]

# Create dictionary mapping
warehouse_shipping_cost = shipping_costs_df.set_index(['Warehouse_ID', 'Source_Factory_ID', 'Product_ID'])['Shipping Cost'].to_dict() # (Warehouse, Factory, Product) -> to Shipping Cost
average_pieces = monthly_pieces.set_index(' Source Factory')['Average'].to_dict()  # Factory -> Average Monthly Production
selling_price = products.set_index('Product_ID')['Selling_Price'].to_dict()  # Product -> Selling Price mapping
manufacturing_cost = production_costs.set_index(['Factory_ID', 'Product_ID'])['Manufac_Cost'].to_dict()  # (Factory, Product) -> Manufacturing Cost mapping

# Create Model
model = ConcreteModel()

# Define Sets
model.Products = Set(initialize=unique_products)
model.Factories = Set(initialize=unique_factories)
model.Warehouses = Set(initialize=unique_warehouses)

# Define Variables
model.ProductionTransport = Var(model.Factories, model.Warehouses, model.Products, domain=NonNegativeReals)

# Define Constraints
# Constraint 1: Total production for each factory must be equal to average
model.AverageConstraints = ConstraintList()
for f in model.Factories:
    model.AverageConstraints.add(
        sum(model.ProductionTransport[f, w, p] for w in model.Warehouses for p in model.Products) == average_pieces[f]
    )

# Constraint 2: Total production for each product must meet or exceed its average monthly demand
model.DemandConstraints = ConstraintList()
for p in model.Products:
    demand_avg = demand_df[demand_df['Product_ID'] == p]['Average'].sum()
    model.DemandConstraints.add(
        sum(model.ProductionTransport[f, w, p] for f in model.Factories for w in model.Warehouses) >= demand_avg
    )

# Constraint 3: Total production and transport for each warehouse must not exceed its maximum capacity
model.WarehouseCapacityConstraints = ConstraintList()
for w in model.Warehouses:
    max_capacity = warehouse_monthly_pieces[warehouse_monthly_pieces['Warehouse_ID'] == w]['Maximum'].iloc[0]
    model.WarehouseCapacityConstraints.add(
        sum(model.ProductionTransport[f, w, p] for f in model.Factories for p in model.Products) <= max_capacity
    )

# Define Objective Function (minimizing the cost)
def objective_rule(model):
    return sum(warehouse_shipping_cost[w, f, p] * model.ProductionTransport[f, w, p]
               for f in model.Factories
               for w in model.Warehouses
               for p in model.Products)

model.Objective = Objective(rule=objective_rule, sense=minimize)

# Solve the model
solver = SolverFactory('cbc')
results = solver.solve(model)

# Display results
if results.solver.status == SolverStatus.ok and results.solver.termination_condition == TerminationCondition.optimal:
    print("Optimal solution found")
    for f in model.Factories:
        for w in model.Warehouses:
            for p in model.Products:
                if value(model.ProductionTransport[f, w, p]) > 0:
                    print(f"Transport from factory {f} to warehouse {w} for product {p}: {value(model.ProductionTransport[f, w, p])}")

    print(f"Total shipping cost: {value(model.Objective)}")
else:
    print("No optimal solution found")

Optimal solution found
Transport from factory F001 to warehouse W001 for product P017: 495229.3
Transport from factory F001 to warehouse W004 for product P017: 562589.0
Transport from factory F001 to warehouse W006 for product P008: 343440.6
Transport from factory F001 to warehouse W006 for product P011: 76100.1
Transport from factory F001 to warehouse W006 for product P017: 129242.3
Transport from factory F001 to warehouse W010 for product P021: 20432.7
Transport from factory F001 to warehouse W011 for product P011: 227560.4
Transport from factory F001 to warehouse W011 for product P016: 298697.1
Transport from factory F001 to warehouse W011 for product P024: 50952.5
Transport from factory F001 to warehouse W014 for product P020: 16817.6
Transport from factory F001 to warehouse W016 for product P006: 413367.4
Transport from factory F001 to warehouse W016 for product P011: 92300.6
Transport from factory F001 to warehouse W019 for product P016: 45208.5
Transport from factory F002 to war

The model was able to find an optimal solution and the Total Shipping Cost would be $4.98 Million.

## Production Analysis


In [ ]:
from pyomo.environ import *

# Initialize Data
# Generate unique product IDs (P001 to P040)
unique_products = [f'P{str(i).zfill(3)}' for i in range(1, 41)]
unique_factories = ['F001', 'F002', 'F003', 'F005'] #factory ids we are using

# Create dictionary mapping
selling_price = products.set_index('Product_ID')['Selling_Price'].to_dict()  # Product -> Selling Price mapping
manufacturing_cost = production_costs.set_index(['Factory_ID', 'Product_ID'])['Manufac_Cost'].to_dict()  # (Factory, Product) -> Manufacturing Cost mapping
average_pieces = monthly_pieces.set_index(' Source Factory')['Average'].to_dict() # Factory -> Average monthly pieces sold

In [ ]:
# Create and Initialize model
model = ConcreteModel()

# Define Sets for factories and products
model.Factories = Set(initialize=unique_factories)
model.Products = Set(initialize=unique_products)

# Define Decision Variables
# Production quantity for each factory-product pair
model.Production = Var(model.Factories, model.Products, domain=NonNegativeReals)

# Define Constraints
# Constraint 1: Total production for each factory must not exceed its average
model.AverageConstraints = ConstraintList()
for f in unique_factories:
    model.AverageConstraints.add(
        sum(model.Production[f, p] for p in unique_products) <= average_pieces[f]
    )

# Constraint 2: Total production for each product must meet or exceed its average monthly demand
model.DemandConstraints = ConstraintList()
for p in unique_products:
    # Get the average monthly demand for the product
    demand_avg = demand_df[demand_df['Product_ID'] == p]['Average'].sum()

    # Ensure total production across all factories for this product meets demand
    model.DemandConstraints.add(
        sum(model.Production[f, p] for f in unique_factories) <= demand_avg
    )

In [ ]:
# Define Objective Function
# Objective: Maximize Profit
def profit_rule(model):
    # Total sales: Sum of (produced quantity * selling price)
    total_sales = sum(
        model.Production[f, p] * selling_price[p]
        for f in model.Factories
        for p in model.Products
        if p in selling_price
    )
    # Total production cost: Sum of (produced quantity * manufacturing cost)
    total_cost = sum(
        model.Production[f, p] * manufacturing_cost.get((f, p), 0)
        for f in model.Factories
        for p in model.Products
    )
    # Profit = Total Sales - Total Cost
    return total_sales - total_cost

# Add the objective function to the model
model.Profit = Objective(rule=profit_rule, sense=maximize)

# Solve the Model
solver = SolverFactory('cbc')
results = solver.solve(model)

# Solver status and termination condition
print("Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

# Display Optimal Production Plan
# Create an empty DataFrame to store the production matrix
production_matrix = pd.DataFrame(
    0.0, index=unique_factories, columns=unique_products
)

# Populate the matrix with values from the decision variables
for f in unique_factories:
    for p in unique_products:
        value = model.Production[f, p].value
        if value is not None:  # Ensure the variable has a value
            production_matrix.loc[f, p] = value


# Display Total Profit
print("\nTotal Profit:", model.Profit())

Status: ok
Termination Condition: optimal

Total Profit: 204597388.9832638


The model was able to find an optimal solution and the Total Profit would be $204.6 Million.

In [ ]:
# Display production matrix for clarity
production_matrix

,P001,P002,P003,P004,P005,P006,P007,P008,P009,P010,...,P031,P032,P033,P034,P035,P036,P037,P038,P039,P040
F001,0.0,0.0,0.0,369989.9,390390.9,0.0,0.0,0.0,0.0,335945.9,...,0.0,17688.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F002,467387.3,415176.0,0.0,0.0,0.0,0.0,0.0,0.0,365950.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20171.0
F003,0.0,0.0,390391.0,0.0,0.0,0.0,0.0,343440.6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F005,0.0,0.0,0.0,0.0,0.0,413367.4,370886.4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# save the matrix to a CSV file
production_matrix.to_csv('production_matrix.csv', index=True)